#Game room data

In [ ]:
#TODO change to game room data
# define rooms and items

couch = {
    "name": "couch",
    "type": "furniture",
}

door_a = {
    "name": "door a",
    "type": "door",
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

piano = {
    "name": "piano",
    "type": "furniture",
}

game_room = {
    "name": "game room",
    "type": "room",
}

bedroom_1 = {
  "name": "bedroom 1"
}

all_rooms = [game_room, bedroom_1]

all_doors = [door_a]

# define which items/rooms are related



# Bedroom 1 data

In [ ]:


queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}

door_a = {
    "name": "door a",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}



bedroom_1 = {
    "name": "bedroom 1",
    "type": "room",
}

outside = {
  "name": "outside"
}

available = [bedroom_1, outside]

all_doors = [door_a, door_b, door_c]

# define which items/rooms are related




#Bedroom 2 data

In [ ]:
#TODO change to bedroom 2 data
# define rooms and items

double_bed = {
    "name": "double bed",
    "type": "furniture",
}

door_b = {
    "name": "door b",
    "type": "door",
}
door_d = {
    "name": "door d",
    "type": "door",
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}
key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}
dresser = {
    "name": "dresser",
    "type": "furniture",
}

bedroom_2 = {
    "name": "bedroom 2",
    "type": "room",
}

outside = {
  "name": "outside"
}

all_rooms = [game_room, outside]

all_doors = [door_a]

# define which items/rooms are related





# Living Room data

In [ ]:
#TODO change to living room data
# define rooms and items

dining_table = {
    "name": "dining table",
    "type": "furniture",
}

door_c = {
    "name": "door c",
    "type": "door",
}
door_d = {
    "name": "door d",
    "type": "door",
}

living_room = {
    "name": "living room",
    "type": "room",
}

outside = {
  "name": "outside"
}

all_rooms = [game_room, outside]

all_doors = [door_a]

# define which items/rooms are related


INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [ ]:
object_relations = {
    "game room": [couch, piano, door_a],
    "piano": [key_a],
    "door a": [game_room, bedroom_1],
    "bedroom 1": [queen_bed, door_a, door_b, door_c],
    "queen bed": [key_b],
    "door c": [bedroom_1, living_room],
    "bedroom 2": [double_bed, dresser, door_b],
    "double bed": [key_c],
    "door b": [bedroom_1, bedroom_2],
    "living room": [dining_table, door_c, door_d],
    "outside": [door_d],
    "door d": [living_room, outside],
    "dresser": [key_d]
}

#Functions Definitions


Functions for Game actions

In [ ]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:

        if(not current_room == room):
            return room

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def navigate_room(current_room, intended_action):
    """
    Handle room navigation. Check if the intended action is valid for the current room.
    Return the next room or None if the action is invalid.
    """
    next_room = None
    if intended_action == "explore":
        explore_room(current_room)
    elif intended_action == "examine":
        item_name = input("What would you like to examine?").strip()
        examine_result = examine_item(item_name)
        if examine_result:
            if examine_result["type"] == "door" and examine_result["unlocked"]:
                next_room = get_next_room_of_door(examine_result, current_room)
                print("You unlock the door and proceed to the next room.")
            elif examine_result["type"] == "door" and not examine_result["unlocked"]:
                print("The door is locked, and you need a key to unlock it.")
            elif examine_result["type"] == "item":
                game_state["keys_collected"].append(examine_result)
                print(f"You find {examine_result['name']}.")
    else:
        print("Not sure what you mean. Type 'explore' or 'examine'.")
    return next_room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")

    if(next_room and input("Do you want to go to the next room? Enter 'yes' or 'no'").strip() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)

def is_door_unlocked(door):
    """
    Check if a door is unlocked (the player has the key for it).
    """
    for key in game_state["keys_collected"]:
        if key["target"] == door:
            return True
    return False

def play_room(room):
    game_state["current_room"] = room
    if room == game_state["target_room"]:
        print("Congrats! You escaped the room!")

    else:
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip()
        next_room = navigate_room(room, intended_action)
        if next_room:
            play_room(next_room)
        else:
            play_room(room)
        linebreak()



def start_game():
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    play_room(game_state["current_room"])



game_state = INIT_GAME_STATE.copy()

#MAIN LOOP( TO SIMULATE GAME)
def main():
  start_game()
  while game_state["current_room"]!=game_state["target_room"]:

    if is_door_unlocked(game_state["current_room"]) is True:
      play_room(game_state["next_room"])
    else:
      play_room(game_state["current_room"])







# Main Game Loop

In [ ]:
main()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!
You are now in game room
